In [ ]:
# {
#   "by": "css",                     // "xpath", "css", "id", "name", etc.
#   "value": "your-selector-here",  // Selector string
#   "wait": 0.5,                    // (Optional) seconds to wait before doing anything
#   "wait_until": "visible",        // "clickable", "visible", "present", etc.
#   "timeout": 10,                  // Max time to wait for `wait_until` condition

#   "action": "click",              // "click", "scrape", "check", "screenshot", etc.
#   "log_message": "Describe this action clearly",  // Log for debugging

#   "new_window": false,            // (Optional) If this click opens new tab
#   "return_to_base": false,        // (Optional) If true, go back to base window after action

#   "multiple": false,              // (Only for scrape) True to loop over many matches

#   "scrape_fields": {              // (Only for scrape) Define keys and child-selectors
#     "title": "h3 a",
#     "price": "p.price_color"
#   },

#   "attribute": null,              // (Only for scrape) If scraping attribute instead of text

#   "check_text": null,             // (Optional) For action: "check" — verify expected text
#   "screenshot_path": null         // (Optional) For action: "screenshot" — save path
# }


In [6]:
import time, base64, os, re,json5, pprint
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
import pandas as pd
import numpy as np

In [ ]:
import json5
from dateutil.parser import parse
import re

path = r'example.json5'

def extract_date(text: str) -> str:
    if not isinstance(text, str):
        print("Invalid data type. Expected string.")
        return ""
    
    output_format = "%Y%m%d"
    date_pattern = r"(\d{2}[.\-/]+\d{2}[.\-/]+\d{4}|\d{1,2}\s*(?:th|st|rd|nd)\s*[A-Za-z]+\s*\d{4}|\d{2}[\.\-\/]+[A-Za-z]+[\.\-\/]+\d{4}|\d{2}\s*[A-Za-z]+\s*\d{4})"
    matches = re.findall(date_pattern, text, re.IGNORECASE)
    
    if matches:
        date_str = " ".join(matches)
        dt_object = parse(date_str, fuzzy=True)
        return dt_object.strftime(output_format)
    
    # print("Date not found.")
    return text

with open(path, "r+", encoding="utf-8") as file:
    data = json5.load(file)

records = data.get("records", [])

for record in records:
    
    print(record["bank_name"])
    response_data = record.get("response", {})
    if not response_data:
        continue

    extracted_dates = {}

    for block in response_data.values():
        scrape_data = block.get("scrape_data", {})
        if not scrape_data:
            continue

        cleaned_data = {key: extract_date(value) for key, value in scrape_data.items()}
        extracted_dates.update(cleaned_data)

    print(extracted_dates)
    record["response"] = extracted_dates

with open("up20250801.json5", "r+", encoding="utf-8") as file:
   json5.dump(data,file)

In [45]:
#BANK OF BARODA
options = Options()
# options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = "https://www.bankofbaroda.in/interest-rate-and-service-charges/deposits-interest-rates"
driver = webdriver.Chrome(options=options)
driver.get(path)
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "component-content"))
)

# headers = driver.find_elements(By.CSS_SELECTOR,".field-heading")

tables = driver.find_elements(By.CSS_SELECTOR,".table-resopnsive")

# for header in headers:
#     print(header.text)

for idx,content in enumerate(tables):
    
    tables = content.find_elements(By.TAG_NAME,"table")
    for table in tables:
        data = table.get_attribute("outerHTML")
        data = re.sub(r'\t+', '', data)
        with open(f"example_{idx}.html","w+", encoding="utf8") as file:
            file.write(data)
    # pprint.pprint(data)
# driver.quit()


#other data
# path = 'https://www.bankofbaroda.in/interest-rate-and-service-charges/retail-loans-interest-rates'
# driver.get(path)

# WebDriverWait(driver, 5).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "col-sm-12"))
# )

# dater = driver.find_element(By.CSS_SELECTOR,"h4+ p")
# print(dater.text)

time.sleep(10)
driver.quit()

In [22]:
#BANK OF INDIA
options = Options()
# options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")
path = 'https://bankofindia.co.in/interest-rate/saving-bank-deposit-rate'

driver = webdriver.Chrome(options=options)
driver.get(path)

WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "card-body"))
)

elements = driver.find_elements(By.CSS_SELECTOR,".th-exchange+ .th-exchange")

for elem in elements:
    print(f">> {elem.text}")

>> REVISED RATE OF INTEREST (% PER ANNUM)
W.E.F. 07.07.2025


In [19]:
options = Options()
# options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = 'https://canarabank.com/pages/deposit-interest-rates'
driver = webdriver.Chrome(options=options)
driver.get(path)

WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.CLASS_NAME, "main-content"))
)

main_content = driver.find_element(By.CLASS_NAME,"main-content")

tables = main_content.find_elements(By.TAG_NAME,'table')
print(len(tables))

ALLOWED = {"rowspan", "colspan"}

for idx, table in enumerate(tables):
    html_str = table.get_attribute("outerHTML").encode().decode()
    soup = BeautifulSoup(html_str, "html.parser")

    for tag in soup.find_all(True):
        # keep only the span attributes
        tag.attrs = {k: v for k, v in tag.attrs.items() if k in ALLOWED}

    clean_html = str(soup)
    with open(f"example_{idx}.html", "w+", encoding="utf-8") as f:
        f.write(clean_html)

# savings = driver.find_elements(By.CSS_SELECTOR,".text-sec > p:nth-child(1) strong")
# bulk_callable = driver.find_elements(By.CSS_SELECTOR,"p:nth-child(8) b")
# bulk_non_callable = driver.find_elements(By.CSS_SELECTOR,"p:nth-child(11) b")
# fncr_deposits = driver.find_elements(By.CSS_SELECTOR,".MsoNormal span")
driver.quit()

9


In [34]:
df = pd.read_html("example_1.html")[0]
df = df.map(lambda x: re.sub(r"[^A-Za-z0-9\s\.\-\+]+","",str(x).strip(), re.IGNORECASE))
df = df.map(lambda x: re.sub(r"\s+"," ",str(x).strip(), re.IGNORECASE))
df.to_excel("sample_1.xlsx",index=False)

C:\Users\kaustubh.keny\AppData\Local\Temp\ipykernel_19668\3875827870.py:2: DeprecationWarning: 'count' is passed as positional argument
  df = df.map(lambda x: re.sub(r"[^A-Za-z0-9\s\.\-\+]+","",str(x).strip(), re.IGNORECASE))
C:\Users\kaustubh.keny\AppData\Local\Temp\ipykernel_19668\3875827870.py:3: DeprecationWarning: 'count' is passed as positional argument
  df = df.map(lambda x: re.sub(r"\s+"," ",str(x).strip(), re.IGNORECASE))


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

# Example: Get page source using Selenium
driver = webdriver.Chrome()
driver.get("https://example.com")
html = driver.page_source

# Parse with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Remove all attributes from tags
for tag in soup.find_all(True):  # True matches all tags
    tag.attrs = {}

# Get cleaned HTML
clean_html = str(soup)
print(clean_html)

In [4]:
import time
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options



#setup
driver = webdriver.Chrome() #controls the entire session
driver.get("https://ucobank.com/interest-rates-on-deposit-schemes")

#wait
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "mainbody"))
)

#toggle/click
toggles = driver.find_elements(By.CLASS_NAME, "collapsed")
for toggle in toggles:
    driver.execute_script("arguments[0].click();", toggle)  # safer than .click()
    time.sleep(0.5)

#tables
tables = driver.find_elements(By.CLASS_NAME, "table-responsive")

all_data = []
for table in tables:
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text.strip() for cell in cells]
        if row_data:
            all_data.append(row_data)

driver.quit()

#save
df = pd.DataFrame(all_data)
df.to_excel("interest_rates.xlsx", index=False, header=False)


In [ ]:
#INDIAN BANK

options = Options()
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = "https://www.indianbank.in/departments/deposit-rates/#!"

driver = webdriver.Chrome()
driver.get(path)

WebDriverWait(driver,5).until(
    EC.presence_of_element_located((By.CLASS_NAME,"container"))
)

rates = driver.find_element(By.CSS_SELECTOR, "tr+ .specificBackground td:nth-child(2) > strong")

# for rate in rates:
#     print("Start",rate.get_attribute("outerHTML"),"End")
print(rates.text)
driver.quit()

Revised w.e.f 09.06.2025


In [29]:
#PNB BANK

options = Options()
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36")

path = "https://www.pnbindia.in/Interest-Rates-Deposit.html"

driver = webdriver.Chrome()
driver.get(path)

# element = driver.find_element(By.CSS_SELECTOR, "#fa-tab1321 table tbody tr:nth-child(2) th:last-of-type p")

entire_html = driver.find_element(By.CSS_SELECTOR,".financials-archives")

tables = entire_html.find_elements(By.TAG_NAME,"table")
print(len(tables))

with open("pnb.html","w+",encoding="utf-8") as f:
    f.write(entire_html.get_attribute("innerHTML"))
# print(element.text)
# print(element.get_attribute("innerHTML"), type(element.get_attribute("innerHTML")))

24
